# Lesson 4: Persistence and Streaming

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_tavily import TavilySearch


In [4]:
tool = TavilySearch(max_results=2)

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)

memory = SqliteSaver(conn=conn)


In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [19]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o-mini")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [20]:
messages = [HumanMessage(content="What is the weather in Saigon?")]

In [21]:
thread = {"configurable": {"thread_id": "1"}}

In [22]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gy4KY27DxPYWBaeXlLp6sTB6', 'function': {'arguments': '{"query":"current weather Saigon","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1341, 'total_tokens': 1364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C28zFNq6qxHq6zqiCKbORtRK6LTvD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9a47dfcf-78c4-43ab-82f4-bbb86100efb9-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather Saigon', 'topic': 'general'}, 'id': 'call_gy4KY27DxPYWBaeXlLp6sTB6', 'type': 'tool_call'}], usage_metadata={'input_to

In [23]:
messages = [HumanMessage(content="What about in Hanoi?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2y3Nt7NUEFK8KxfdcMLqLIBh', 'function': {'arguments': '{"query":"current weather Hanoi","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 2116, 'total_tokens': 2138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2048}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C2910HISpqsKAI0yjk92DCPqK7LCe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4babec6d-4899-4ba0-ad66-9218cf0c075d-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather Hanoi', 'topic': 'general'}, 'id': 'call_2y3Nt7NUEFK8KxfdcMLqLIBh', 'type': 'tool_call'}], usage_metad

In [24]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Currently, Saigon is slightly warmer than Hanoi:\n\n- **Saigon**: 33.2°C (feels like 36.3°C)\n- **Hanoi**: 33.1°C (feels like 40.7°C)\n\nAlthough Hanoi has a similar temperature, it feels significantly warmer due to higher humidity.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 2834, 'total_tokens': 2903, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2688}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C291Iy7h8zR3ZF3PJVzQFQsu5pMdr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--322618f5-7b24-4a44-a4d5-db9b9e6baa55-0', usage_metadata={'input_tokens': 2834, 'output_tokens': 69, 'total_tokens': 2903, 'input_token_details': {'audio': 0, 'cache_read': 26

In [25]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please provide more context? Are you comparing two specific locations, objects, or time periods? Let me know, and I can assist you better!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1338, 'total_tokens': 1370, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C291uAn5sh0M0rohgl7J9RiNfzVsB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7dac81c3-518a-4275-9af2-beb34f15df16-0', usage_metadata={'input_tokens': 1338, 'output_tokens': 32, 'total_tokens': 1370, 'input_token_details': {'audio': 0, 'cache_read': 1280}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


## Streaming tokens

In [35]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
import sqlite3

conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
memory = AsyncSqliteSaver(conn=conn)



In [47]:

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [52]:
import nest_asyncio
nest_asyncio.apply()

messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async with AsyncSqliteSaver.from_conn_string("checkpoints.sqlite") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")

Calling: {'name': 'tavily_search', 'args': {'query': 'San Francisco weather'}, 'id': 'call_14yUdl0YRJro6fFRRl2Q011Q', 'type': 'tool_call'}
Back to the model!
The| current| weather| in| San| Francisco| is| partly| cloudy| with| a| temperature| of| |15|.|6|°C| (|60|.|1|°F|).| The| wind| is| coming| from| the| west|-s|outh|west| at| |9|.|4| mph|,| and| the| humidity| level| is| |86|%.| 

|For| more| detailed| or| updated| weather| information|,| you| can| visit| [|Weather| API|](|https|://|www|.weather|api|.com|/|).|

In [49]:
import asyncio

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph

async def main():
    builder = StateGraph(int)
    builder.add_node("add_one", lambda x: x + 1)
    builder.set_entry_point("add_one")
    builder.set_finish_point("add_one")
    async with AsyncSqliteSaver.from_conn_string("checkpoints.db") as memory:
        graph = builder.compile(checkpointer=memory)
        coro = graph.ainvoke(1, {"configurable": {"thread_id": "thread-1"}})
        print(await asyncio.gather(coro))

asyncio.run(main())

[2]
